We started by building intuition for model based learning, explored how the linear regression model worked, understood how the two different approaches to model fitting worked, and some techniques for cleaning, transforming, and selecting features.

In this project, we will practice what we learned by exploring ways to improve the models we built. We'll work with housing data for the city of Ames, Iowa, United States from 2006 to 2010.

In [360]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [361]:
df = pd.read_csv("AmesHousing.txt", delimiter="\t")

In [362]:
def transform_features(df):
    return df

def select_features(df):
    return df[["Gr Liv Area", "SalePrice"]]

def train_and_test(df):  
    train = df[:1460]
    test = df[1460:]
    numeric_train = train.select_dtypes(include = ["integer", "float"])
    numeric_test = test.select_dtypes(include = ["integer", "float"])
    
    features = numeric_train.columns.drop("SalePrice")
    target = "SalePrice"
    
    lr = LinearRegression()
    lr.fit(train[features], train[target])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test[target], predictions)
    rmse = np.sqrt(mse)
    return rmse

In [363]:
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse

57088.25161263909

# Feature Engineering

* Handle missing values:
 * All columns:
      * Drop any with 5% or more missing values for now.
 * Text columns:
      * Drop any with 1 or more missing values for now.
 * Numerical columns:
      * For columns with missing values, fill in with the most common value in that column
 

In [364]:
# 1. All columns: Drop any with 5% or more missing values for now.

num_missing = df.isnull().sum()

drop_missing_cols = num_missing[num_missing > len(df)/20].sort_values()

df = df.drop(drop_missing_cols.index, axis = 1)


In [365]:
# 2: Text columns: Drop any with 1 or more missing values for now.

text_mv_counts = df.select_dtypes(include = ["object"]).isnull().sum().sort_values(ascending = False)

drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]
df = df.drop(drop_missing_cols_2.index, axis = 1)

In [366]:
# 3: Numerical columns: For columns with missing values, fill in with the most common value in that column

num_missing = df.select_dtypes(include = ["integer", "float"]).isnull().sum().sort_values()
num_missing

Order               0
Mo Sold             0
Misc Val            0
Pool Area           0
Screen Porch        0
3Ssn Porch          0
Enclosed Porch      0
Open Porch SF       0
Wood Deck SF        0
Fireplaces          0
TotRms AbvGrd       0
Kitchen AbvGr       0
Bedroom AbvGr       0
Half Bath           0
Full Bath           0
Yr Sold             0
SalePrice           0
MS SubClass         0
Low Qual Fin SF     0
2nd Flr SF          0
1st Flr SF          0
Overall Qual        0
Overall Cond        0
Year Built          0
Year Remod/Add      0
Lot Area            0
Gr Liv Area         0
PID                 0
Garage Area         1
BsmtFin SF 2        1
Bsmt Unf SF         1
Total Bsmt SF       1
BsmtFin SF 1        1
Garage Cars         1
Bsmt Full Bath      2
Bsmt Half Bath      2
Mas Vnr Area       23
dtype: int64

In [367]:
# Compute the most common value for each column in `fixable_nmeric_missing_cols`.

replacement_values_dict = df[num_missing.index].mode().to_dict(orient = "record")[0]
replacement_values_dict

{'Order': 1.0,
 'Mo Sold': 6.0,
 'Misc Val': 0.0,
 'Pool Area': 0.0,
 'Screen Porch': 0.0,
 '3Ssn Porch': 0.0,
 'Enclosed Porch': 0.0,
 'Open Porch SF': 0.0,
 'Wood Deck SF': 0.0,
 'Fireplaces': 0.0,
 'TotRms AbvGrd': 6.0,
 'Kitchen AbvGr': 1.0,
 'Bedroom AbvGr': 3.0,
 'Half Bath': 0.0,
 'Full Bath': 2.0,
 'Yr Sold': 2007.0,
 'SalePrice': 135000.0,
 'MS SubClass': 20.0,
 'Low Qual Fin SF': 0.0,
 '2nd Flr SF': 0.0,
 '1st Flr SF': 864.0,
 'Overall Qual': 5.0,
 'Overall Cond': 5.0,
 'Year Built': 2005.0,
 'Year Remod/Add': 1950.0,
 'Lot Area': 9600.0,
 'Gr Liv Area': 864.0,
 'PID': 526301100.0,
 'Garage Area': 0.0,
 'BsmtFin SF 2': 0.0,
 'Bsmt Unf SF': 0.0,
 'Total Bsmt SF': 0.0,
 'BsmtFin SF 1': 0.0,
 'Garage Cars': 2.0,
 'Bsmt Full Bath': 0.0,
 'Bsmt Half Bath': 0.0,
 'Mas Vnr Area': 0.0}

In [368]:
df = df.fillna(replacement_values_dict)

In [369]:
# Verify that every column has 0 missing values
df.isnull().sum().value_counts()

0    64
dtype: int64

### What new features can we create, that better capture the information in some of the features?

In [370]:
years_sold = df['Yr Sold'] - df['Year Built']
years_sold[years_sold < 0]

2180   -1
dtype: int64

In [371]:
years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
years_since_remod[years_since_remod < 0]

1702   -1
2180   -2
2181   -1
dtype: int64

In [372]:
## Create new columns
df['Years Before Sale'] = years_sold
df['Years Since Remod'] = years_since_remod

df = df.drop([1702,2180,2181])

df.drop(["Year Built", "Year Remod/Add"], axis = 1, inplace = True)

Drop columns that:

* that aren't useful for ML
* leak data about the final sale, read more about columns here

In [373]:
## Drop columns that aren't useful for ML
df = df.drop(["PID", "Order"], axis=1)

## Drop columns that leak info about the final sale
df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)

In [374]:
def transform_features(df):
    num_missing = df.isnull().sum()
    drop_missing_cols = num_missing[num_missing > len(df)/20].sort_values()
    df = df.drop(drop_missing_cols.index, axis = 1)
    
    text_mv_counts = df.select_dtypes(include = ["object"]).isnull().sum().sort_values(ascending = False)
    drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]
    df = df.drop(drop_missing_cols_2.index, axis = 1)
    
    num_missing = df.select_dtypes(include = ["integer", "float"]).isnull().sum().sort_values()
    replacement_values_dict = df[num_missing.index].mode().to_dict(orient = "record")[0]
    df = df.fillna(replacement_values_dict)
    
    years_sold = df['Yr Sold'] - df['Year Built']
    years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
    df['Years Before Sale'] = years_sold
    df['Years Since Remod'] = years_since_remod
    df = df.drop([1702,2180,2181])
    df.drop(["Year Built", "Year Remod/Add"], axis = 1, inplace = True)
    df = df.drop(["PID", "Order"], axis=1)
    df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)
    return df

def select_features(df):
    return df[["Gr Liv Area", "SalePrice"]]

def train_and_test(df):  
    train = df[:1460]
    test = df[1460:]
    numeric_train = train.select_dtypes(include = ["integer", "float"])
    numeric_test = test.select_dtypes(include = ["integer", "float"])
    
    features = numeric_train.columns.drop("SalePrice")
    target = "SalePrice"
    
    lr = LinearRegression()
    lr.fit(train[features], train[target])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test[target], predictions)
    rmse = np.sqrt(mse)
    return rmse



In [375]:
df = pd.read_csv("AmesHousing.txt", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse

55275.367312413066

# Feature Selection

In [376]:
numerical_df = df.select_dtypes(include = ["integer", "float"])
numerical_df.shape

(2930, 39)

In [377]:
# Which features correlate strongly with our target column, SalePrice

abs_corr_coeffs = numerical_df.corr()["SalePrice"].abs().sort_values()
abs_corr_coeffs

BsmtFin SF 2       0.005891
Misc Val           0.015691
Yr Sold            0.030569
Order              0.031408
3Ssn Porch         0.032225
Mo Sold            0.035259
Bsmt Half Bath     0.035835
Low Qual Fin SF    0.037660
Pool Area          0.068403
MS SubClass        0.085092
Overall Cond       0.101697
Screen Porch       0.112151
Kitchen AbvGr      0.119814
Enclosed Porch     0.128787
Bedroom AbvGr      0.143913
Bsmt Unf SF        0.182855
PID                0.246521
Lot Area           0.266549
2nd Flr SF         0.269373
Bsmt Full Bath     0.276050
Half Bath          0.285056
Open Porch SF      0.312951
Wood Deck SF       0.327143
Lot Frontage       0.357318
BsmtFin SF 1       0.432914
Fireplaces         0.474558
TotRms AbvGrd      0.495474
Mas Vnr Area       0.508285
Garage Yr Blt      0.526965
Year Remod/Add     0.532974
Full Bath          0.545604
Year Built         0.558426
1st Flr SF         0.621676
Total Bsmt SF      0.632280
Garage Area        0.640401
Garage Cars        0

In [378]:
# Drop columns with less than 0.4 correlation with SalePrice
transform_df = numerical_df.drop(abs_corr_coeffs[abs_corr_coeffs>0.4].index, axis = 1)

Which categorical columns should we keep?

In [379]:
# Create a list of column names from documentation that are *meant* to be categorical
nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

* Which columns are currently numerical but need to be encoded as categorical instead (because the numbers don't have any semantic meaning)?
* If a categorical column has hundreds of unique values (or categories), should we keep it? When we dummy code this column, hundreds of columns will need to be added back to the data frame.

In [380]:
# Which categorical columns have we still carried with us? We'll test these 
transform_cat_cols = []
for col in nominal_features:
    if col in transform_df.columns:
        transform_cat_cols.append(col)

transform_cat_cols
        

['PID', 'MS SubClass']

In [381]:
## How many unique values in each categorical column?
uniqueness_counts = transform_df[transform_cat_cols].apply(lambda col:len(col.value_counts())).sort_values()
uniqueness_counts

MS SubClass      16
PID            2930
dtype: int64

In [382]:
# Aribtrary cutoff of 10 unique values (worth experimenting)

drop_nonuniq_cols = uniqueness_counts[uniqueness_counts>10].index
drop_nonuniq_cols

Index(['MS SubClass', 'PID'], dtype='object')

In [383]:
transform_df = transform_df.drop(drop_nonuniq_cols, axis = 1)
transform_df.head(2)

,Order,Lot Frontage,Lot Area,Overall Cond,BsmtFin SF 2,Bsmt Unf SF,2nd Flr SF,Low Qual Fin SF,Bsmt Full Bath,Bsmt Half Bath,...,Kitchen AbvGr,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold
0,1,141.0,31770,5,0.0,441.0,0,0,1.0,0.0,...,1,210,62,0,0,0,0,0,5,2010
1,2,80.0,11622,6,144.0,270.0,0,0,0.0,0.0,...,1,140,0,0,0,120,0,0,6,2010


In [384]:
## Select just the remaining text columns and convert to categorical

text_cols  = transform_df.select_dtypes(include = ["object"])

for col in text_cols:
    transform_df[col] = transform_df[col].astype("category")
    transform_df = pd.concat([transform_df, pd.get_dummies(transform_df[col])], axis = 1)
    transform_df.drop(col, axis = 1)

In [385]:
# or 
# transform_df = pd.concat([
#     transform_df, 
#     pd.get_dummies(transform_df.select_dtypes(include=['category']))
# ], axis=1).drop(text_cols,axis=1)

In [412]:
def transform_features(df):
    num_missing = df.isnull().sum()
    drop_missing_cols = num_missing[(num_missing > len(df)/20)].sort_values()
    df = df.drop(drop_missing_cols.index, axis=1)
    
    text_mv_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]
    df = df.drop(drop_missing_cols_2.index, axis=1)
    
    num_missing = df.select_dtypes(include=['integer', 'float']).isnull().sum()
    fixable_numeric_cols = num_missing[(num_missing < len(df)/20) & (num_missing > 0)].sort_values()
    replacement_values_dict = df[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replacement_values_dict)
    
    years_sold = df['Yr Sold'] - df['Year Built']
    years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
    df['Years Before Sale'] = years_sold
    df['Years Since Remod'] = years_since_remod
    df = df.drop([1702, 2180, 2181], axis=0)

    df = df.drop(["PID", "Order", "Mo Sold", "Sale Condition", "Sale Type", "Year Built", "Year Remod/Add"], axis=1)
    return df

def select_features(df, coeff_threshold=0.4, uniq_threshold=10):
    numerical_df = df.select_dtypes(include=['integer', 'float'])
    abs_corr_coeffs = numerical_df.corr()["SalePrice"].abs().sort_values()
    df = df.drop(abs_corr_coeffs[abs_corr_coeffs < coeff_threshold].index, axis=1)
    
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    
    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)

    uniqueness_counts = df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
    drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
    df = df.drop(drop_nonuniq_cols, axis=1)
    
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))], axis=1).drop(text_cols,axis=1)
    
    return df

def train_and_test(df, k=0):
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.drop("SalePrice")
    lr = linear_model.LinearRegression()
    
    if k == 0: #  holdout validation
        train = df[:1460]
        test = df[1460:]

        lr.fit(train[features], train["SalePrice"])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test["SalePrice"], predictions)
        rmse = np.sqrt(mse)

        return rmse
    
    if k == 1: # simple cross validation
        # Randomize *all* rows (frac=1) from `df` and return
        shuffled_df = df.sample(frac=1, )
        train = df[:1460]
        test = df[1460:]
        
        lr.fit(train[features], train["SalePrice"])
        predictions_one = lr.predict(test[features])        
        
        mse_one = mean_squared_error(test["SalePrice"], predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        lr.fit(test[features], test["SalePrice"])
        predictions_two = lr.predict(train[features])        
       
        mse_two = mean_squared_error(train["SalePrice"], predictions_two)
        rmse_two = np.sqrt(mse_two)
        
        avg_rmse = np.mean([rmse_one, rmse_two])
        print(rmse_one)
        print(rmse_two)
        return avg_rmse
    else: # k-fold cross validation
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train["SalePrice"])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test["SalePrice"], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse


In [413]:
df = pd.read_csv("AmesHousing.txt", delimiter="\t")
transform_df = transform_features(df)

In [414]:
filtered_df = select_features(transform_df)

In [415]:
rmse = train_and_test(filtered_df, k=4)

rmse

[27861.068279070976, 36822.61967732247, 24554.4623122184, 26208.279216455805]


28861.607371266913